Baseado na análise exploratória pretendo extrair informações dos documentos da seguinte maneira:

- Fazer o preprocessamento de dados desenvolvido na etapa de análise exploratória com alguns novos ajustes
- Utilizar um modelo Bert do tipo Biobert_pt, Um modelo de linguagem clínica para português (treinado com EHR de hospitais), para nomeação de entidades  [[Repositório]](https://github.com/HAILab-PUCPR/BioBERTpt). O repositório contém modelos de BERT ajustados e treinados no domínio clínico para a língua portuguesa. BERT-multilingual-cased pré-treinados foram ajustados com narrativas clínicas de hospitais brasileiros e resumos de artigos científicos do Pubmed e Scielo.
- Não irei remover as stopwords uma vez que o modelo foi treinado com frases completas, incluindo as stopwords, e essas palavras podem ser importantes para manter o contexto e a estrutura do texto.

## Imports

In [3]:
import pandas as pd
import torch
import numpy as np
import json

import re
import nltk    
from nltk import tokenize 
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForTokenClassification

from pathlib import Path
from tqdm import tqdm
import warnings

## Configs

In [5]:
warnings.filterwarnings('ignore')

raw_data_path = Path('../00_dados/raw/sample_teste_nlp.csv')
bert_model_path = Path('../models/biobert-all-clinpt')

## Funções auxiliares

In [7]:
def processar_texto(texto):
    """
    Processa um texto aplicando uma série de etapas para limpeza e normalização,
    incluindo remoção de formatações HTML e RTF, correção de codificação de caracteres,
    remoção de stopwords e filtro de palavras específicas ('none', 'false', 'true').
    
    Parâmetros:
    - texto (str): O texto a ser processado.

    Retorna:
    - str: Texto limpo, corrigido e filtrado.
    """
    
    # 1. Limpeza de formatações HTML e RTF
    texto = BeautifulSoup(texto, "html.parser").get_text()  # Remove tags HTML
    texto = re.sub(r'\\[a-z]+[0-9]*', '', texto)           # Remove tags RTF como \ansi, \fonttbl, etc.
    texto = re.sub(r'{\\.*?}', '', texto)                  # Remove grupos RTF como {\rtf1...}
    texto = re.sub(r'[{}\[\]]', '', texto)                 # Remove colchetes e chaves do RTF {, }, [, ]
    texto = re.sub(r'\r\n|\r|\n', ' ', texto)              # Substitui quebras de linha por espaço
    
    # 2. Remoção de fontes e padrões desnecessários
    fontes = r'\b(?:arial|wingdings|monospaced|courier|times new roman|verdana|' \
             r'calibri|microsoft sans serif|unicode ms|default)\b'
    texto = re.sub(fontes, '', texto, flags=re.IGNORECASE)
    
    # 3. Normalização de caracteres e conversão para minúsculas
    texto = re.sub(r'[^a-zA-Z0-9\sáéíóúâêîôûãõçÁÉÍÓÚÂÊÎÔÛÃÕÇ]', '', texto)  # Remove caracteres especiais
    texto = texto.lower().strip()  # Converte para minúsculas e remove espaços desnecessários
    
    # 4. Correção de codificação de caracteres
    mapeamento = {
        'c3': 'ã', 'c7': 'ç', 'e7': 'ç', 'e3': 'ã', 'f3': 'ó', 'c9': 'é',
        'e9': 'é', 'e1': 'á', 'c1': 'á', 'e2': 'â', 'c2': 'â', 'f5': 'ô'
    }
    for padrao, caractere in mapeamento.items():
        texto = re.sub(padrao, caractere, texto, flags=re.IGNORECASE)
    
    # Remove números isolados e espaços extras
    texto = re.sub(r'\b\d+\b', '', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()

    # 5. Filtro de palavras específicas ('none', 'false', 'true')
    palavras_filtrar = {'none', 'false', 'true'}
    tokens = [word for word in texto.split() if word not in palavras_filtrar]
    
    return ' '.join(tokens)


def aplicar_bert_ner(df, coluna_texto, caminho_modelo_bert, max_len=512, device=None):
    """
    Aplica o modelo BERT para realizar NER (Reconhecimento de Entidades Nomeadas) em uma coluna de texto de um DataFrame,
    com suporte a CUDA (GPU) se disponível.

    Parâmetros:
    -----------
    df : pandas.DataFrame
        DataFrame que contém a coluna de texto para análise.
    coluna_texto : str
        O nome da coluna do DataFrame que contém as sentenças de texto.
    caminho_modelo_bert : str ou Path
        Caminho para o diretório onde o modelo BERT pré-treinado e os arquivos auxiliares estão armazenados.
    max_len : int
        Número máximo de tokens que o modelo BERT pode processar (limite de 512).
    device : torch.device
        O dispositivo (CPU ou GPU) onde o modelo e os dados serão processados.
        
    Retorna:
    --------
    pandas.DataFrame
        O DataFrame original com uma nova coluna contendo as predições de NER.
    """
    
    # Configura o dispositivo
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Carrega o modelo e o tokenizer BERT
    modelo = BertForTokenClassification.from_pretrained(caminho_modelo_bert)
    modelo.to(device)  # Move o modelo para o dispositivo escolhido
    tokenizer = BertTokenizer.from_pretrained(caminho_modelo_bert, do_lower_case=True)
    
    # Carrega o mapeamento de índices para etiquetas
    with open(caminho_modelo_bert / 'idx2tag.json', 'r') as arquivo:
        idx2tag = json.load(arquivo)
    
    # Lista para armazenar as predições
    predicoes = []
    
    # Adiciona a barra de progresso com tqdm no DataFrame
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processando linhas", unit="linha"):
        frase = row[coluna_texto]
        
        # Tokeniza a frase e cria chunks com comprimento máximo de `max_len`
        tokens = tokenizer.tokenize(frase)
        chunks = [tokens[i:i + max_len - 2] for i in range(0, len(tokens), max_len - 2)]
        
        rotulos_finais = []
        
        for chunk in chunks:
            # Adicionar tokens especiais [CLS] e [SEP]
            input_ids = tokenizer.encode(chunk, add_special_tokens=True)
            input_tensor = torch.tensor([input_ids]).to(device)  # Move os dados de entrada para o dispositivo
            
            # Realiza a predição sem calcular gradientes
            with torch.no_grad():
                saida = modelo(input_tensor)
            
            # Identifica os índices das etiquetas preditas
            indices_etiquetas = np.argmax(saida[0].cpu().numpy(), axis=2)
            
            # Converte os tokens e remove tokens divididos por sub-palavras
            tokens = tokenizer.convert_ids_to_tokens(input_tensor.cpu().numpy()[0])
            novos_tokens, novas_etiquetas = [], []
            for token, idx_etiqueta in zip(tokens, indices_etiquetas[0]):
                if token.startswith("##"):
                    novos_tokens[-1] = novos_tokens[-1] + token[2:]  # Remove prefixo de sub-palavra
                else:
                    novas_etiquetas.append(idx_etiqueta)
                    novos_tokens.append(token)
            
            # Converte os índices de etiquetas para etiquetas textuais e filtra tokens irrelevantes
            for token, etiqueta in zip(novos_tokens, novas_etiquetas):
                etiqueta = idx2tag[str(etiqueta)]
                if etiqueta in ["O", "X"]:
                    rotulos_finais.append("O")
                else:
                    rotulos_finais.append(etiqueta)
        
        # Armazena as predições finais da sentença completa, removendo [CLS] e [SEP]
        predicoes.append(rotulos_finais[1:-1])
    
    # Cria uma nova coluna no DataFrame com as predições
    df['predicoes_ner'] = predicoes
    
    return df

## Importação dos Dados

In [9]:
df = pd.read_csv(raw_data_path)
df.head()

,patient_id,age,sex,ehr_date,text
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:..."
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:..."
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...


## Preprocessamento dos dados

In [11]:
df['clean_text'] = df['text'].apply(processar_texto)

In [12]:
df.head()

,patient_id,age,sex,ehr_date,text,clean_text
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:...",anamnese centro clínico queixa principal dor i...
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:...",paciente com cefaleia occipital crise há dias ...
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,anotação de enfermagem recebo plantão em posto...
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D,ret meses para avaliação calcio vit d
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,evolução de enfermagem conferido pulseira de i...


## Estruturação das entidades

In [14]:
df_processado = aplicar_bert_ner(df, 'clean_text', bert_model_path)

Processando linhas: 100%|███████████████████████████████████████████████████████| 6834/6834 [02:24<00:00, 47.27linha/s]


In [23]:
df_processado

,patient_id,age,sex,ehr_date,text,clean_text,predicoes_ner
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:...",anamnese centro clínico queixa principal dor i...,"[B-T, O, O, O, O, B-C, B-AS, I-AS, B-CH, B-C, ..."
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:...",paciente com cefaleia occipital crise há dias ...,"[O, O, B-C, B-AS, B-DT, B-DT, I-DT, B-C, I-C, ..."
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,anotação de enfermagem recebo plantão em posto...,"[O, O, O, O, B-OBS, O, O, O, O, O, B-OBS, O, O..."
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D,ret meses para avaliação calcio vit d,"[B-EV, B-DT, O, B-T, I-T, I-T, I-T]"
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,evolução de enfermagem conferido pulseira de i...,"[B-EV, O, B-T, O, B-THER, I-THER, I-T, O, B-C,..."
...,...,...,...,...,...,...,...
6829,4888904,40.0,M,2023-11-17 18:23:20,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,anotação de enfermagem realizado medicação seg...,"[I-DT, O, O, O, B-THER, O, B-THER, I-THER, I-T..."
6830,395277,38.0,F,2023-09-27 22:15:42,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,evolução médica recebo resultado de exame urin...,"[B-EV, I-EV, O, O, O, B-T, I-T, B-R, I-R, I-R,..."
6831,2086843,61.0,M,2023-11-08 19:19:44,"<html tasy=""html5""><body><p style=""text-align:...",anotação de enfermagem recebo plantão paciente...,"[O, O, O, O, O, O, O, O, O, O, B-OBS, O, B-OBS..."
6832,500002679850091,37.0,F,2020-12-28 00:00:00,"# NOME: BRUNA ALVES PINTO PEREIRA, 32 ANOS, DN...",nome bruna alves pinto pereira anos dn estado ...,"[O, B-OBS, I-OBS, I-OBS, I-OBS, I-DT, O, B-OBS..."


Agora nós temos as frases e suas tags, veja o exemplo:

In [36]:
text = df_processado.clean_text[0]
text

'anamnese centro clínico queixa principal dor inguinoescrotal esq ae alergias nega alergia medicamentos hda hpp sinais vitaispa x fc fr t dor exame físico hipótese diagnósticahernia inguinal condutaexames'

In [63]:
labels = df_processado.predicoes_ner[0]
str(labels)

"['B-T', 'O', 'O', 'O', 'O', 'B-C', 'B-AS', 'I-AS', 'B-CH', 'B-C', 'B-N', 'B-THER', 'I-THER', 'O', 'B-R', 'B-R', 'I-R', 'I-R', 'I-R', 'I-R', 'I-R', 'I-R', 'B-T', 'I-T', 'B-CH', 'I-CH', 'B-AS', 'O']"

In [61]:
for n, i in enumerate(text.split()):
    print(i, "---> ", labels[n])

anamnese --->  B-T
centro --->  O
clínico --->  O
queixa --->  O
principal --->  O
dor --->  B-C
inguinoescrotal --->  B-AS
esq --->  I-AS
ae --->  B-CH
alergias --->  B-C
nega --->  B-N
alergia --->  B-THER
medicamentos --->  I-THER
hda --->  O
hpp --->  B-R
sinais --->  B-R
vitaispa --->  I-R
x --->  I-R
fc --->  I-R
fr --->  I-R
t --->  I-R
dor --->  I-R
exame --->  B-T
físico --->  I-T
hipótese --->  B-CH
diagnósticahernia --->  I-CH
inguinal --->  B-AS
condutaexames --->  O


O modelo BioBERT aplicou tags que indicam o início (B) ou a continuidade (I) de entidades específicas dentro do texto médico, como sintomas (dor), histórico clínico (hipótese, diagnósticos), sinais vitais (fc, fr), e tratamentos/condutas (alergias, medicamentos). As tags O indicam partes do texto que o modelo não reconheceu como entidades clínicas relevantes. <br>
<br>
anamnese --->  B-T, Begin test, começo do teste.<br>
dor --->  B-C, begin clinical situation, a dor começou.<br>
alergia --->  B-THER, begin therapy, inicio do tratamento, o modelo entendeu que o inicio do tratamento começou aqui.<br>
medicamentos --->  I-THE, in therapy, em tratamento, indica que o paciente ainda está em tratamento.<br>
hpp --->  B-R, begin results, inicio dos resultados, o modelo entendeu que o inicio dos resultados estão aqui.<br>
sinais --->  B-, begin results, inicio dos resultados, o modelo entendeu que o inicio dos resultados estão aqui.<br>R
vitaispa --->  I in results, inicio dos resultados, o modelo entendeu que aqui estão dentro dos resultados.<br>-R
x --->   in results, inicio dos resultados, o modelo entendeu que aqui estão dentro dos resultados.<br>
hipótese --->  B-CH, begin clinal history, começo dos histórico clínico<br>
diagnósticahernia --->  I-C, in clinal history, dentro histórico cl<br>

O modelo não conseguiu captar tudo perfeitamente, mas é possível fazer o fine tunning desse modelo, que é opensourse, dessa forma ter resultados ainda mais precHI-RR

# Próximos Passos: Análise de Prontuários com BioBERT

Com as predições realizadas pelo **BioBERT**, é possível extrair informações valiosas do prontuário médico e realizar análises mais profundas para melhorar o acompanhamento clínico. Abaixo estão algumas estratégias e sugestões de como utilizar as entidades preditas para obter insights mais completos dos prontuários.

## 1. Extração de Entidades Clínicas

Com as predições, você pode identificar e extrair entidades clínicas importantes, como:

- **Sintomas**: Identifique sintomas relatados, como "dor", "febre", etc.
- **Diagnósticos**: Encontre diagnósticos clínicos, como "hipótese diagnóstica: hérnia inguinal".
- **Medicações**: Extraia informações sobre medicamentos prescritos.
- **Histórico Médico**: Obtenha histórico de doenças prévias, alergias e outras condições clínicas.
- **Sinais Vitais**: Extraia dados de sinais vitais como pressão arterial, frequência cardíaca, etc.
- **Exames e Procedimentos**: Identifique exames realizados ou procedimentos recomendados.

### Exemplo:
- Sintoma: **dor inguinoescrotal** (B-C, B-AS)
- Diagnóstico: **hipótese diagnóstica: hérnia inguinal** (B-CH, I-CH)

## 2. Identificação de Padrões e Análise de Tendências por sexo e idade

Utilizando as entidades extraídas, é possível identificar padrões e tendências em diferentes pacientes ou grupos. Algumas análises incluem:

- **Sintomas mais recorrentes**: Identificar os sintomas mais comuns em grupos de pacientes.
- **Tratamentos comuns**: Quais tratamentos estão associados a diagnósticos como "hernia inguinal"?
- **Padrões de sinais vitais**: Como os sinais vitais variam para condições específicas?

## 3. Criação de Resumos Automáticos do Prontuário

Você pode gerar resumos automatizados de prontuários clínicos, com informações organizadas e facilmente acessíveis:

- **Histórico médico** (ex: doenças prévias, alergias)
- **Diagnóstico atual**
- **Sintomas e sinais vitais**
- **Tratamentos recomendados**
- **Exames solicitados**

## 4. Identificação de Relações entre Entidades

A análise das relações entre as entidades extraídas pode fornecer insights valiosos. Algumas possíveis relações incluem:

- **Diagnóstico e tratamento**: Quais tratamentos estão associados a quais diagnósticos?
- **Histórico de doenças e medicações**: Quais condições prévias estão associadas a certos medicamentos?
- **Sinais vitais e condições clínicas**: Como os sinais vitais estão relacionados aos sintomas e diagnósticos?

## 5. Classificação e Agrupamento de Pacientes

Com base nas entidades extraídas, é possível agrupar os pacientes de acordo com características clínicas. Alguns exemplos de agrupamento:

- **Grupo 1**: Pacientes com histórico de hérnia inguinal.
- **Grupo 2**: Pacientes com sintomas de dor inguinoescrotal.
- **Grupo 3**: Pacientes com alergias a medicamentos.

Essa segmentação pode ser utilizada para personalizar tratamentos e melhorar o monitoramento dos pacientes.

## 6. Detecção de Falhas ou Omissões nos Prontuários

Através das entidades extraídas, é possível detectar falhas ou omissões nos prontuários médicos. Exemplos incluem:

- **Ausência de informações importantes**: Faltando histórico médico ou exames realizados.
- **Inconsistências em tratamentos ou diagnósticos**: Diagnósticos e tratamentos não correspondem aos sintomas.
- **Faltando diagnósticos**: Sintomas não têm diagnóstico associado.

## 7. Extração de Dados para Acompanhamento Longitudinal

Com as entidades extraídas, é possível construir registros para o **acompanhamento longitudinal** dos pacientes:

- **Mudanças nos sintomas**: Como os sintomas evoluem ao longo do tempo?
- **Alterações nos sinais vitais**: Como a pressão arterial ou a frequência cardíaca variam durante o tratamento?

## Como Implementar

1. **Criação de Funções de Extração**: Com base nas predições, crie funções para extrair e organizar as entidades em uma estrutura mais acessível, como DataFrames ou tabelas em banco de dados.
   
2. **Análise de Dados**: Realize análises descritivas ou preditivas sobre as entidades extraídas. Busque por tendências, padrões e relações entre as entidades.

3. **Visualização**: Utilize ferramentas de visualização para mapear as relações entre sintomas, diagnósticos e tratamentos, como gráficos de dispersão, redes de relações e análises temporais.

## Exemplos de Ações:

### Extração de Sintoma e Diagnóstico
- Sintomas: **Dor inguinoescrotal**, **dor**.
- Diagnóstico: **Hernia inguinal**.

### Construção de Banco de Dados Estruturado
- Organize as entidades extraídas para análises quantitativas, como **frequência de sintomas**, **tratamento recomendado**, etc.

### Análise de Padrões e Agrupamento
- Use técnicas de clustering para segmentar os pacientes com base em sintomas semelhantes, idade, sexo e tratamentos correlacionados.